## Machine Learning in .NET with ML .NET
Because of course you can do machine learning in .NET

### Setup

This project relies on ML .NET and its Auto ML module. Both will be installed from NuGet Package Manager.

In [1]:
#r "nuget:Microsoft.ML"
#r "nuget:Microsoft.ML.AutoML"

Installed Packages Microsoft.ML, 2.0.1 Microsoft.ML.AutoML, 0.20.1

Loading extensions from `C:\Users\MattE\.nuget\packages\microsoft.ml.automl\0.20.1\interactive-extensions\dotnet\Microsoft.ML.AutoML.Interactive.dll`

Loading extensions from `C:\Users\MattE\.nuget\packages\skiasharp\2.88.3\interactive-extensions\dotnet\SkiaSharp.DotNet.Interactive.dll`

In [2]:
// All the imports we'll need
using Microsoft.ML;
using Microsoft.ML.AutoML;
using Microsoft.ML.Data;

using System.Collections.Generic;

In [3]:
// Everything in ML .NET revolves around a context object
MLContext Context = new();

### Load Datasets

In [4]:
#!import 04_GameInfo.cs

GameInfo sampleGame = new() {
    Title = "Stir Trek 2023 - the Game",
    Language = true, // Python, C#, JS, etc.
    AlcoholReference = true, // this is it
};
sampleGame

Title,Stir Trek 2023 - the Game
Console,False
AlcoholReference,True
AnimatedBlood,False
Blood,False
BloodAndGore,False
CartoonViolence,False
CrudeHumor,False
DrugReference,False
FantasyViolence,False
IntenseViolence,False


In [5]:
// Load training data. Taken from Kaggle dataset at https://www.kaggle.com/imohtn/video-games-rating-by-esrb
IDataView trainData = Context.Data.LoadFromTextFile<GameInfo>(
    path: "ESRB.csv",
    separatorChar: ',',
    hasHeader: true,
    allowQuoting: true);

In [6]:
// Load test data. Taken from Kaggle dataset at https://www.kaggle.com/imohtn/video-games-rating-by-esrb
IDataView validationData = Context.Data.LoadFromTextFile<GameInfo>(
    path: "ESRBTest.csv", 
    separatorChar: ',', 
    hasHeader: true, 
    allowQuoting: true);

### Training
This uses automated machine learning to find the best featurization and training settings for the dataset

In [7]:
uint secondsToTrain = 10;

// Configure the experiment
MulticlassClassificationExperiment experiment = 
    Context.Auto().CreateMulticlassClassificationExperiment(secondsToTrain);

In [8]:
// Synchronously Train the model
ExperimentResult<MulticlassClassificationMetrics> result = 
    experiment.Execute(
        trainData: trainData, // The data to memorize
        validationData: validationData, // How we evaluate our model's performance against new data
        labelColumnName: nameof(GameInfo.ESRBRating), // What we're trying to predict
        progressHandler: new MulticlassConsoleProgressReporter()); // How to show progress

ConvertType=>FeaturizeText=>Concatenate=>Unknown=>FastTreeOva=>Unknown ran in 1.16 seconds with accuracy of 75.41%
ConvertType=>FeaturizeText=>Concatenate=>Unknown=>FastTreeOva=>Unknown ran in 1.09 seconds with accuracy of 74.23%
ConvertType=>FeaturizeText=>Concatenate=>Unknown=>FastTreeOva=>Unknown ran in 0.79 seconds with accuracy of 71.23%
ConvertType=>FeaturizeText=>Concatenate=>Unknown=>FastTreeOva=>Unknown ran in 1.16 seconds with accuracy of 73.83%
ConvertType=>FeaturizeText=>Concatenate=>Unknown=>LbfgsMaximumEntropyMulti=>Unknown ran in 0.50 seconds with accuracy of 76.97%
ConvertType=>FeaturizeText=>Concatenate=>Unknown=>LightGbmMulti=>Unknown ran in 0.38 seconds with accuracy of 74.65%
ConvertType=>FeaturizeText=>Concatenate=>Unknown=>SdcaMaximumEntropyMulti=>Unknown ran in 0.32 seconds with accuracy of 55.29%
ConvertType=>FeaturizeText=>Concatenate=>Unknown=>LbfgsMaximumEntropyMulti=>Unknown ran in 0.34 seconds with accuracy of 78.16%
ConvertType=>FeaturizeText=>Concatenate=

In [9]:
// Display details about the result
result

Microsoft.ML.AutoML.ExperimentResult`1[Microsoft.ML.Data.MulticlassClassificationMetrics] RunDetails index value 0 Microsoft.ML.AutoML.RunDetail`1[Microsoft.ML.Data.MulticlassClassificationMetrics] ValidationMetrics Microsoft.ML.Data.MulticlassClassificationMetrics LogLoss 1.1520647996815911 LogLossReduction 0.14393564785032045 MacroAccuracy 0.7541235334713596 MicroAccuracy 0.762 TopKAccuracy 0 TopKPredictionCount 0 TopKAccuracyForAllK <null> PerClassLogLoss [ 1.1427545367264178, 1.16242499424014, 1.2166878715626754, 1.118421132898133 ] ConfusionMatrix Microsoft.ML.Data.ConfusionMatrix PerClassPrecision [ 0.872093023255814, 0.7933884297520661, 0.8421052631578947, 0.6728110599078341 ] PerClassRecall [ 0.75, 0.7619047619047619, 0.7111111111111111, 0.7934782608695652 ] Counts index value 0 [ 75, 4, 0, 21 ] 1 [ 4, 96, 0, 26 ] 2 [ 0, 2, 64, 24 ] 3 [ 7, 19, 12, 146 ] NumberOfClasses 4 Model index type value 0 Microsoft.ML.Transforms.TypeConvertingTransformer Microsoft.ML.Transforms.TypeConvertingTransformer 1 Microsoft.ML.Transforms.Text.TextFeaturizingEstimator+Transformer Microsoft.ML.Transforms.Text.TextFeaturizingEstimator+Transformer 2 Microsoft.ML.Data.ColumnConcatenatingTransformer Microsoft.ML.Data.ColumnConcatenatingTransformer 3 Microsoft.ML.Transforms.ValueToKeyMappingTransformer Microsoft.ML.Transforms.ValueToKeyMappingTransformer 4 Microsoft.ML.Data.MulticlassPredictionTransformer<Microsoft.ML.Trainers.OneVersusAllModelParameters> Microsoft.ML.Data.MulticlassPredictionTransformer`1[Microsoft.ML.Trainers.OneVersusAllModelParameters] FeatureColumnName __Features__ FeatureColumnType Vector Dimensions [ 11774 ] IsKnownSize True ItemType Single Size 11774 RawType Microsoft.ML.Data.VBuffer`1[System.Single] Model Microsoft.ML.Trainers.OneVersusAllModelParameters 5 Microsoft.ML.Transforms.KeyToValueMappingTransformer Microsoft.ML.Transforms.KeyToValueMappingTransformer Exception <null> TrainerName ConvertType=>FeaturizeText=>Concatenate=>Unknown=>LightGbmMulti=>Unknown RuntimeInSeconds 1.164 Estimator Microsoft.ML.Data.EstimatorChain`1[Microsoft.ML.ITransformer] LastEstimator Microsoft.ML.Transforms.KeyToValueMappingEstimator 1 Microsoft.ML.AutoML.RunDetail`1[Microsoft.ML.Data.MulticlassClassificationMetrics] ValidationMetrics Microsoft.ML.Data.MulticlassClassificationMetrics LogLoss 0.658376412058391 LogLossReduction 0.5107804901120416 MacroAccuracy 0.7423171152518978 MicroAccuracy 0.76 TopKAccuracy 0 TopKPredictionCount 0 TopKAccuracyForAllK <null> PerClassLogLoss [ 0.6095828672793098, 0.6719582915735663, 0.9021952290725155, 0.5563348040568951 ] ConfusionMatrix Microsoft.ML.Data.ConfusionMatrix PerClassPrecision [ 0.9120879120879121, 0.7829457364341085, 0.8421052631578947, 0.6636771300448431 ] PerClassRecall [ 0.83, 0.8015873015873016, 0.5333333333333333, 0.8043478260869565 ] Counts index value 0 [ 83, 4, 0, 13 ] 1 [ 5, 101, 0, 20 ] 2 [ 0, 0, 48, 42 ] 3 [ 3, 24, 9, 148 ] NumberOfClasses 4 Model index type value 0 Microsoft.ML.Transforms.TypeConvertingTransformer Microsoft.ML.Transforms.TypeConvertingTransformer 1 Microsoft.ML.Transforms.Text.TextFeaturizingEstimator+Transformer Microsoft.ML.Transforms.Text.TextFeaturizingEstimator+Transformer 2 Microsoft.ML.Data.ColumnConcatenatingTransformer Microsoft.ML.Data.ColumnConcatenatingTransformer 3 Microsoft.ML.Transforms.ValueToKeyMappingTransformer Microsoft.ML.Transforms.ValueToKeyMappingTransformer 4 Microsoft.ML.Data.MulticlassPredictionTransformer<Microsoft.ML.Trainers.OneVersusAllModelParameters> Microsoft.ML.Data.MulticlassPredictionTransformer`1[Microsoft.ML.Trainers.OneVersusAllModelParameters] FeatureColumnName __Features__ FeatureColumnType Vector Dimensions [ 11774 ] IsKnownSize True ItemType Single Size 11774 RawType Microsoft.ML.Data.VBuffer`1[System.Single] Model Microsoft.ML.Trainers.OneVersusAllModelParameters 5 Microsoft.ML.Transforms.KeyToValueMappingTransformer Microsoft.ML.Transforms.KeyToValueMappingTransformer Exception <null> TrainerName ConvertType=>Featur

In [10]:
// Grab the trained model for later
ITransformer model = result.BestRun.Model;

model

index type value 0 Microsoft.ML.Transforms.TypeConvertingTransformer Microsoft.ML.Transforms.TypeConvertingTransformer 1 Microsoft.ML.Transforms.Text.TextFeaturizingEstimator+Transformer Microsoft.ML.Transforms.Text.TextFeaturizingEstimator+Transformer 2 Microsoft.ML.Data.ColumnConcatenatingTransformer Microsoft.ML.Data.ColumnConcatenatingTransformer 3 Microsoft.ML.Transforms.ValueToKeyMappingTransformer Microsoft.ML.Transforms.ValueToKeyMappingTransformer 4 Microsoft.ML.Data.MulticlassPredictionTransformer<Microsoft.ML.Trainers.MaximumEntropyModelParameters> Microsoft.ML.Data.MulticlassPredictionTransformer`1[Microsoft.ML.Trainers.MaximumEntropyModelParameters] FeatureColumnName __Features__ FeatureColumnType Vector Dimensions [ 11774 ] IsKnownSize True ItemType Single RawType System.Single Size 11774 RawType Microsoft.ML.Data.VBuffer<System.Single> Model Microsoft.ML.Trainers.MaximumEntropyModelParameters Statistics <null> 5 Microsoft.ML.Transforms.KeyToValueMappingTransformer Microsoft.ML.Transforms.KeyToValueMappingTransformer

### Model Evaluation

In [11]:
// Identify the best run's algorithm
result.BestRun.TrainerName

ConvertType=>FeaturizeText=>Concatenate=>Unknown=>LightGbmMulti=>Unknown

In [12]:
// Take a look a the best run's metrics
result.BestRun.ValidationMetrics

Microsoft.ML.Data.MulticlassClassificationMetrics LogLoss 0.5119809709411925 LogLossReduction 0.6195624948155082 MacroAccuracy 0.7816295721187025 MicroAccuracy 0.798 TopKAccuracy 0 TopKPredictionCount 0 TopKAccuracyForAllK <null> PerClassLogLoss [ 0.37216390349222467, 0.5097796982759034, 0.9725721810641614, 0.3641867219719299 ] ConfusionMatrix Microsoft.ML.Data.ConfusionMatrix PerClassPrecision [ 0.9375, 0.7984496124031008, 0.8947368421052632, 0.7110091743119266 ] PerClassRecall [ 0.9, 0.8174603174603174, 0.5666666666666667, 0.842391304347826 ] Counts index value 0 [ 90, 5, 0, 5 ] 1 [ 4, 103, 0, 19 ] 2 [ 0, 0, 51, 39 ] 3 [ 2, 21, 6, 155 ] NumberOfClasses 4

In [13]:
result.BestRun.ValidationMetrics.ConfusionMatrix.GetFormattedConfusionTable()


Confusion table
          ||================================
PREDICTED ||     E |    ET |     M |     T | Recall
TRUTH     ||================================
        E ||    90 |     5 |     0 |     5 | 0.9000
       ET ||     4 |   103 |     0 |    19 | 0.8175
        M ||     0 |     0 |    51 |    39 | 0.5667
        T ||     2 |    21 |     6 |   155 | 0.8424
          ||================================
Precision ||0.9375 |0.7984 |0.8947 |0.7110 |


### Generating Predictions

In [14]:
    /// <summary>
    /// This class represents a set of confidences in the various ESRB ratings for a single video game
    /// </summary>
    public class ESRBPrediction
    {
        [ColumnName("PredictedLabel")]
        public string ESRBRating { get; set; }

        [ColumnName("Score")]
        public float[] Score { get; set; }
    }

In [15]:
// Create a PredictionEngine that can generate predictions for games we've not seen before
PredictionEngine<GameInfo, ESRBPrediction> predictor =
    Context.Model.CreatePredictionEngine<GameInfo, ESRBPrediction>(
            transformer: model);

In [16]:
// Create a list of sample games
List<GameInfo> games = new() {
    new GameInfo()
    {
        Title = "Comic Doggo Side Scroller for Teens",
        CartoonViolence = true,
        MildLanguage = true,
        CrudeHumor = true,
        Violence = true,
    },
    new GameInfo()
    {
        Title = "Kinda Sus",
        MildCartoonViolence = true
    },
    new GameInfo()
    {
        Title = "The Earthlings are Coming",
        MildViolence = true,
        MildFantasyViolence = true,
    },
    new GameInfo()
    {
        Title = "Shoddy Surgeon Simulator",
        BloodAndGore = true,
        DrugReference = true,
        PartialNudity = true,
    },
    new GameInfo()
    {
        Title = "Assistant to the Lawn Service Manager 2023",
        MildLanguage = true,
        CrudeHumor = true,
        AlcoholReference = true,
    },
    new GameInfo()
    {
        Title = "Intense Shoot-o-rama: Why would anyone play this edition",
        BloodAndGore = true,
        DrugReference = true,
        AlcoholReference = true,
        Nudity = true,
        StrongLanguage = true,
        SexualContent = true,
        SexualThemes = true,
        MatureHumor = true,
        IntenseViolence = true,
        CrudeHumor = true,
    }
};

games

index value 0 {Comic Doggo Side Scroller for Teens} Title Comic Doggo Side Scroller for Teens Console False AlcoholReference False AnimatedBlood False Blood False BloodAndGore False CartoonViolence True CrudeHumor True DrugReference False FantasyViolence False IntenseViolence False Language False Lyrics False MatureHumor False MildBlood False MildCartoonViolence False MildFantasyViolence False MildLanguage True MildLyrics False MildSuggestiveThemes False MildViolence False NoDescriptors False Nudity False PartialNudity False SexualContent False SexualThemes False SimulatedGambling False StrongLanguage False StrongSexualContent False SuggestiveThemes False UseOfAlcohol False UseOfDrugsAndAlcohol False Violence True ESRBRating <null> 1 {Kinda Sus} Title Kinda Sus Console False AlcoholReference False AnimatedBlood False Blood False BloodAndGore False CartoonViolence False CrudeHumor False DrugReference False FantasyViolence False IntenseViolence False Language False Lyrics False MatureHumor False MildBlood False MildCartoonViolence True MildFantasyViolence False MildLanguage False MildLyrics False MildSuggestiveThemes False MildViolence False NoDescriptors False Nudity False PartialNudity False SexualContent False SexualThemes False SimulatedGambling False StrongLanguage False StrongSexualContent False SuggestiveThemes False UseOfAlcohol False UseOfDrugsAndAlcohol False Violence False ESRBRating <null> 2 {The Earthlings are Coming} Title The Earthlings are Coming Console False AlcoholReference False AnimatedBlood False Blood False BloodAndGore False CartoonViolence False CrudeHumor False DrugReference False FantasyViolence False IntenseViolence False Language False Lyrics False MatureHumor False MildBlood False MildCartoonViolence False MildFantasyViolence True MildLanguage False MildLyrics False MildSuggestiveThemes False MildViolence True NoDescriptors False Nudity False PartialNudity False SexualContent False SexualThemes False SimulatedGambling False StrongLanguage False StrongSexualContent False SuggestiveThemes False UseOfAlcohol False UseOfDrugsAndAlcohol False Violence False ESRBRating <null> 3 {Shoddy Surgeon Simulator} Title Shoddy Surgeon Simulator Console False AlcoholReference False AnimatedBlood False Blood False BloodAndGore True CartoonViolence False CrudeHumor False DrugReference True FantasyViolence False IntenseViolence False Language False Lyrics False MatureHumor False MildBlood False MildCartoonViolence False MildFantasyViolence False MildLanguage False MildLyrics False MildSuggestiveThemes False MildViolence False NoDescriptors False Nudity False PartialNudity True SexualContent False SexualThemes False SimulatedGambling False StrongLanguage False StrongSexualContent False SuggestiveThemes False UseOfAlcohol False UseOfDrugsAndAlcohol False Violence False ESRBRating <null> 4 {Assistant to the Lawn Service Manager 2023} Title Assistant to the Lawn Service Manager 2023 Console False AlcoholReference True AnimatedBlood False Blood False BloodAndGore False CartoonViolence False CrudeHumor True DrugReference False FantasyViolence False IntenseViolence False Language False Lyrics False MatureHumor False MildBlood False MildCartoonViolence False MildFantasyViolence False MildLanguage True MildLyrics False MildSuggestiveThemes False MildViolence False NoDescriptors False Nudity False PartialNudity False SexualContent False SexualThemes False SimulatedGambling False StrongLanguage False StrongSexualContent False SuggestiveThemes False UseOfAlcohol False UseOfDrugsAndAlcohol False Violence False ESRBRating <null> 5 {Intense Shoot-o-rama: Why would anyone play this edition} Title Intense Shoot-o-rama: Why would anyone play this edition Console False AlcoholReference True AnimatedBlood False Blood False BloodAndGore True CartoonViolence False CrudeHumor True DrugReference True FantasyViolence False IntenseViolence True Language False Lyrics False MatureHumor True MildBlood False MildCartoonViolence False MildFantasyVio

In [17]:
foreach (GameInfo game in games) {
    ESRBPrediction prediction = predictor.Predict(game);

    Console.WriteLine($"Predicted {prediction.ESRBRating} for {game.Title}");

    Console.Write("   Probabilities (E/ET/M/T): ");
    foreach (float probability in prediction.Score) {
        Console.Write($"{probability:P} ");
    }
    Console.WriteLine();
    Console.WriteLine();
}

Predicted T for Comic Doggo Side Scroller for Teens
   Probabilities (E/ET/M/T): 0.74% 17.54% 0.45% 81.27% 

Predicted E for Kinda Sus
   Probabilities (E/ET/M/T): 61.41% 21.90% 3.46% 13.24% 

Predicted E for The Earthlings are Coming
   Probabilities (E/ET/M/T): 61.40% 20.12% 2.04% 16.44% 

Predicted M for Shoddy Surgeon Simulator
   Probabilities (E/ET/M/T): 3.46% 4.13% 58.03% 34.38% 

Predicted ET for Assistant to the Lawn Service Manager 2023
   Probabilities (E/ET/M/T): 11.74% 54.60% 0.97% 32.69% 

Predicted M for Intense Shoot-o-rama: Why would anyone play this edition
   Probabilities (E/ET/M/T): 0.01% 0.67% 95.39% 3.92% 



### Save the Trained Model for Later

Because training models takes time, we can save the resulting model to a Zip file. This file can then be deployed elsewhere.

In [18]:
Context.Model.Save(model, inputSchema:null, "Model.zip");

Console.WriteLine("Model saved");

Model saved


### Load the Model

Most of the time you just want to use a trained model, not retrain it every time. This is how you'd do that.

In [19]:
ITransformer loadedModel = Context.Model.Load("Model.zip", out _);

loadedModel // After this, we'd build a PredictionEngine

index type value 0 Microsoft.ML.Transforms.TypeConvertingTransformer Microsoft.ML.Transforms.TypeConvertingTransformer 1 Microsoft.ML.Transforms.Text.TextFeaturizingEstimator+Transformer Microsoft.ML.Transforms.Text.TextFeaturizingEstimator+Transformer 2 Microsoft.ML.Data.ColumnConcatenatingTransformer Microsoft.ML.Data.ColumnConcatenatingTransformer 3 Microsoft.ML.Transforms.ValueToKeyMappingTransformer Microsoft.ML.Transforms.ValueToKeyMappingTransformer 4 Microsoft.ML.Data.MulticlassPredictionTransformer<Microsoft.ML.Trainers.MaximumEntropyModelParameters> Microsoft.ML.Data.MulticlassPredictionTransformer`1[Microsoft.ML.Trainers.MaximumEntropyModelParameters] FeatureColumnName __Features__ FeatureColumnType Vector Dimensions [ 11774 ] IsKnownSize True ItemType Single RawType System.Single Size 11774 RawType Microsoft.ML.Data.VBuffer<System.Single> Model Microsoft.ML.Trainers.MaximumEntropyModelParameters Statistics <null> 5 Microsoft.ML.Transforms.KeyToValueMappingTransformer Microsoft.ML.Transforms.KeyToValueMappingTransformer

---

**Note:** models can also be saved to / loaded from ONNX format and used in other machine learning platforms and frameworks.